In [91]:
import numpy as np
import csv
from sklearn import neighbors, datasets, model_selection, metrics, __version__
from math import log2
import pandas as pd
import os

## Creating the the different functions.

### starting with the tree class

In [92]:
#Simple tree structure with maximum 2 children
class Tree:
    #Containing a value, either a lable or tuple with information about split. Then each child is a new tree object.
    def __init__(self, value=None):
        self.value = value
        self.left = None
        self.right = None
        self.major_label = None
        
    

### Split labels
Making a function to split a list of labels on its mean. The split function is too simple, because it would split it on the datapoint with the mean.
I could sort the array, but I find it more simple and effective to retrieve the indexes instead and slice the points from the labels.

In [93]:
def split_label(feature, labels):
    #Finding feature mean
    feature_mean = np.nanmean(feature)
    #Splitting the feature on the mean
    left_labels = labels[np.nonzero(feature < feature_mean)]
    right_labels = labels[np.nonzero(feature >= feature_mean)]
    #np.nonzero function finds the index in an array with a condition. Here the conditions are based on the mean.
    return left_labels, right_labels, feature_mean
    

### Impurity and investigation score

In [300]:
def impurity(arr, impurity_measure = 'entropy'):
    #Starting sum
    entropy = 0
    gini = 0
    #Finding the counts for each possible label. Eihter one or two labels. 
    labels, counts = np.unique(arr, return_counts=True)
    #Calculating the entropy and gini for the array
    for count in counts:
        prob = count/len(arr)
        entropy -= prob*log2(prob)
        gini += prob*(1-prob)

    if impurity_measure == 'entropy':
        return entropy
    elif impurity_measure == 'gini':
        return gini

In [301]:
def cond_impurity(feature, labels, impurity_measure='entropy'):
    #Getting the labels for the split
    left_labels, right_labels, f_mean = split_label(feature, labels)
    
    #Finding the count of labels on each side
    left_len = len(left_labels)/len(feature)
    right_len = len(right_labels)/len(feature)

    #Calculating the impurity measure for the feature and each split, and multiplying with the split ratio
    impurity_left = impurity(left_labels, impurity_measure)*left_len
    impurity_right = impurity(right_labels, impurity_measure)*right_len

    #returning the conditional entropy
    return impurity_left + impurity_right

In [302]:
def investigation_score(feature, labels, impurity_measure='entropy'):

    #Getting the entropies
    entropy_feature = impurity(labels, impurity_measure)
    cond_entropy_feature = cond_impurity(feature, labels, impurity_measure)

    #Returning the investigation score
    return entropy_feature - cond_entropy_feature
    

### Finding the best feature to split on, by investigation score

In [303]:
def find_best_feature(data, labels, impurity_measure = 'entropy'):
    #Making a dictionary to store the feature key and the inv_score as value. 
    best_i = {}
    #Iterating through each feature
    for feature in range(data.shape[1]):
        best_i[feature] = investigation_score(data[:, feature], labels, impurity_measure)

    #Finding best feature with max function, where the key is the values. 
    #print(best_i.values())
    #if max(best_i.values()) > 0:
    best_feature_index = max(best_i, key= best_i.get)
    return best_feature_index

### Splitting the data and labels into subarrays
I chose to return a triple containing the left side, right side and value for the node

In [304]:
def split_data(data, labels, impurity_measure = 'entropy', feature_index='best', feature_mean='best'):
    
    #If we want to find best_feature or use a predetermined
    if feature_index == 'best':
        feature_index = find_best_feature(data, labels, impurity_measure)
        best_feature_mean = 'best'

    #Get the feature
    best_feature = data[:, feature_index]

    #Splitting the labels | Could have returned the labels in investigation_score to save compute, but this should be marginal.
    left_labels, right_labels, best_feature_mean = split_label(best_feature, labels)  
    
    #Splitting the data based on the indexes of what points is lower or higher than mean, given specific feature
    left_data = data[best_feature < best_feature_mean]
    right_data = data[best_feature >= best_feature_mean]
    
    #Returning a 3-tuple consisting of the left side, the right side and information about the split (what feature, the mean)
    #The latter will be stored in each branch when building the tree. 
    return (left_data, left_labels), (right_data, right_labels), (feature_index, best_feature_mean)
    

### Checking for identical features
The approach is if there is only one unique value in every column, then the multi array has identical rows. 

In [305]:
def identical_features(data, labels):
    count = 0
    for feature in range(data.shape[1]):
        if len(np.unique(data[:,feature])) == 1:
            count += 1
    
    if count == data.shape[1]:
        uniques, counts = np.unique(labels, return_counts=True)
        return uniques[np.argmax(counts)]

## Implementing the ID3 function by using the prior built functions


In [306]:
def id3(data, labels, tree, impurity_measure = 'entropy'):
    #Finding checking for identical features
    identical = identical_features(data, labels)
    
    #If all data points have the same label:
    if impurity(labels) == 0:
        tree.value = labels[0]
        return labels[0]

    #Else if all data points have identical feature values
    elif identical != None:
        tree.value = identical
        return


    #Else
    else:
        #Extracting the information from the split
        left, right, root = split_data(data, labels, impurity_measure)

        if root != 0:
            #Setting this root to indicate the split
            tree.value = root

            #Setting the majority label
            lab, counts = np.unique(labels, return_counts=True)
            maj_index = np.where(counts == max(counts))[0][0]
            tree.majority_label = lab[maj_index]
    
            #Making left branch
            new_left = Tree()
            tree.left = new_left
            id3(left[0], left[1], new_left, impurity_measure)
    
            #Making right branch
            new_right = Tree()
            tree.right = new_right 
            id3(right[0], right[1], new_right, impurity_measure)
        
        
        
    

## Creating some functions to inspect the tree

In [307]:
def search_tree(tree,count=0):
    if tree != None:
        counts = count
        counts += 1
        print(tree.value)
    
        search_tree(tree.left,counts)
        search_tree(tree.right,counts)

In [308]:
def total_nodes(tree):
    if tree == None:
        return 0

    l = total_nodes(tree.left)
    r = total_nodes(tree.right)

    return 1 + l + r
    

## Prediction and accuracy functions

In [309]:
def predict(data_point, tree):
    if type(tree.value) == tuple:
        feature, split_point = tree.value
        if data_point[feature] < split_point:
            return predict(data_point, tree.left)
        else:
            return predict(data_point, tree.right)
    else:
        return tree.value
    #print(tree.value)
    
    

In [310]:
def accuracy(data, labels, tree):
    labels_len = len(labels)
    if labels_len == 0:
        return 0
    count = 0
    for counts, data_point in enumerate(data):
        if predict(data_point, tree) == labels[counts]:
            count += 1
    return count/labels_len

In [311]:
def predict_flat_label(data, labels, prediction=0):
    labels_len = len(labels)
    if labels_len == 0:
        return 0
    count = 0
    for counts, data_points in enumerate(data):
        if prediction == labels[counts]:
            count += 1
    return count/labels_len

## Pruning algorithm
In essence a Depth first search. Since I implemented the tree structure to have a left and right child, I dont have a child list. But it works the same since when using a for loop on a list of children, you start with the first one and its first one etc. So I'm doing left side first, then calling right side when the left is searched. Instead of for loop im just using running it recusively as long as the child is a tree object. Then on each child im calculating if the accuracy is higher with a lable instead of a split, and if so, the new value is a label.

If there is an empty array, then I'm not changing its branch. This is because the pruning data does not have the same data points as the training data. But when you calculate the accuracy and predictions with labels, they are all 0%. So the pruning data is not really suitable to determine wheter this split is necesarry or not. I have done some observations with different seeds and on average the accuracy declines when pruning these empty arrays. Hence im leaving the branches untouched.  

In [312]:
def prune(data, labels, tree, impurity_measure = 'entropy'):
    #Just want to do the pruning on a node that is splitting, not on a label node.
    if type(tree.value) == tuple:
        feature, mean = tree.value

        #Incase there are empty branches. If the the data and labels are empty, then there is no reason to split it, so the datapoints can stay as they were.
        #Since the accuracy is 0 with given data, but also 0 for either 0 or 1, so it would be unfair to choose one, because there is not enough data in pruning, to make a good decision.
        if len(labels) == 0:
            return
        else:
            left, right, values = split_data(data, labels, impurity_measure, feature_index = feature)
        
        
        
        #If there is a left 
        if type(tree.left) == Tree:
            prune(left[0], left[1], tree.left)
        if type(tree.right) == Tree:
            prune(right[0], right[1], tree.right)

        #Accuracy of either splitting or giving hard label
        for label in np.unique(labels):
            if predict_flat_label(data, labels, prediction=tree.majority_label) > accuracy(data, labels, tree):
                #print('true')
                #print(tree.value, left[1], right[1], predict_flat_label(data, labels, prediction=label), accuracy(data, labels, tree))
                tree.value = tree.majority_label
                tree.left, tree.right = None, None
                #print(tree.value)

    

# The main learning algorithm

In [313]:
#Using the id3 algorithm to return a decision tree.
def learn(X, y, impurity_measure='entropy', pruning="false", ratio=0.8):
    #Making the root for the tree
    tree = Tree()

    #Checking whether the pruning is true
    if pruning == 'True':
        #Now we need to split the data
        ratio = ratio
        X_train, X_prune = np.split(X, [int(ratio*len(X))])
        y_train, y_prune = np.split(y, [int(ratio*len(y))])

        #Making the tree with training data
        id3(X_train, y_train, tree, impurity_measure)
        total_nodes(tree)

        #Pruning the tree with the pruning data
        prune(X_prune, y_prune, tree)
        total_nodes(tree)

        #Returning the pruned tree
        return tree
    
    #Else if pruning is false | Just make the tree and return it.
    else:
        id3(X, y, tree, impurity_measure)
        return tree

### read csv to numpy
Nice to have if we want to load more datasets

In [314]:
def cvs_numpy(name=''):
    with open(name, 'r') as r:
        reader = csv.reader(r)
        data = list(reader)
    
    feature_names = data[0][:-1]
    data_ar = np.array(data[1:], dtype=float)
    targets = data_ar[:, -1]
    data = data_ar[:, :-1]
    return data, targets, feature_names

# Testing withthe wine dataset

In [240]:
data, labels, target_names = cvs_numpy('wine_dataset.csv')#
seed = 521#332#333#521
X_train, X_val_test, y_train, y_val_test = model_selection.train_test_split(data, labels, test_size=0.3, shuffle=True, random_state=seed)

In [241]:
data, labels, target_names

(array([[ 0.13,  1.6 ,  3.34,  0.59,  9.2 ],
        [ 0.1 ,  2.8 ,  3.6 ,  0.66, 10.2 ],
        [ 0.32,  1.9 ,  3.2 ,  0.55,  9.5 ],
        ...,
        [ 0.44,  1.6 ,  3.38,  0.86,  9.9 ],
        [ 0.36,  4.5 ,  3.4 ,  0.57, 10.4 ],
        [ 0.34,  6.4 ,  2.99,  0.4 , 10.8 ]]),
 array([1., 1., 1., ..., 1., 0., 0.]),
 ['citric acid', 'residual sugar', 'pH', 'sulphates', 'alcohol'])

In [242]:
tree = Tree()
tree= learn(X_train, y_train, impurity_measure='entropy')
accuracy(X_train, y_train, tree)

1.0

In [243]:
tree = Tree()
tree= learn(X_train, y_train, impurity_measure='gini', pruning="True", ratio=0.8)

In [244]:
tree.left.left.value
total_nodes(tree)


457

In [245]:
accuracy(X_train, y_train, tree)

0.9436997319034852

In [246]:
accuracy(X_val_test, y_val_test, tree)


0.8875

In [247]:
tree_2 = Tree()
tree_2 = learn(X_train, y_train, impurity_measure='gini')

total_nodes(tree_2), accuracy(X_val_test, y_val_test, tree_2)

(781, 0.8635416666666667)

In [248]:
#prune(X_val_test, y_val_test, tree_2)

In [249]:
total_nodes(tree_2)

781

In [250]:
(accuracy(X_val_test, y_val_test, tree), total_nodes(tree)), (accuracy(X_val_test, y_val_test, tree_2), total_nodes(tree_2))

((0.8875, 457), (0.8635416666666667, 781))

In [251]:
#predict_flat_label(X_val_test, y_val_test, 1)

In [252]:
tree_prunefree = Tree()
tree_prunefree = learn(X_train, y_train, impurity_measure='gini')
accuracy_prunefree = accuracy(X_val_test, y_val_test, tree_prunefree)
print(f'Prune free accuracy: {accuracy_prunefree}')
for x in range(1,10):
    tree = Tree()
    tree = learn(X_train, y_train, impurity_measure='gini', pruning='True', ratio=x/10)
    accuracy_tree = accuracy(X_val_test, y_val_test, tree)
    print(f'Testing ratio: {x/10}, with impurity_measure: gini, accuracy: {accuracy_tree:.3f} | Difference in accuracy: {accuracy_tree- accuracy_prunefree:.5f}')

Prune free accuracy: 0.8635416666666667
Testing ratio: 0.1, with impurity_measure: gini, accuracy: 0.868 | Difference in accuracy: 0.00417
Testing ratio: 0.2, with impurity_measure: gini, accuracy: 0.855 | Difference in accuracy: -0.00833
Testing ratio: 0.3, with impurity_measure: gini, accuracy: 0.872 | Difference in accuracy: 0.00833
Testing ratio: 0.4, with impurity_measure: gini, accuracy: 0.865 | Difference in accuracy: 0.00104
Testing ratio: 0.5, with impurity_measure: gini, accuracy: 0.869 | Difference in accuracy: 0.00521
Testing ratio: 0.6, with impurity_measure: gini, accuracy: 0.876 | Difference in accuracy: 0.01250
Testing ratio: 0.7, with impurity_measure: gini, accuracy: 0.883 | Difference in accuracy: 0.01979
Testing ratio: 0.8, with impurity_measure: gini, accuracy: 0.887 | Difference in accuracy: 0.02396
Testing ratio: 0.9, with impurity_measure: gini, accuracy: 0.884 | Difference in accuracy: 0.02083


In [253]:
search_tree(tree)

(1, 4.56196623634558)
(3, 0.6071335740072202)
(3, 0.4989447236180904)
(0, 0.28191489361702127)
(0, 0.16731250000000003)
(3, 0.436875)
(1, 1.7357142857142855)
(1, 1.3038461538461539)
0.0
(1, 1.5416666666666667)
0.0
(4, 10.766666666666666)
1.0
0.0
(1, 2.4375)
1.0
(0, 0.05500000000000001)
0.0
(0, 0.09)
1.0
0.0
(0, 0.05000000000000001)
1.0
(3, 0.47388888888888886)
(2, 3.2016666666666667)
(0, 0.1)
0.0
1.0
1.0
1.0
(4, 10.594270833333335)
(1, 1.982075471698113)
(1, 1.5716666666666663)
0.0
(0, 0.2426315789473684)
(4, 9.54375)
1.0
(0, 0.21)
1.0
0.0
(2, 3.183636363636364)
0.0
(0, 0.26333333333333336)
1.0
0.0
1.0
(2, 3.1876744186046517)
0.0
(4, 11.7)
0.0
(1, 2.6949999999999994)
(4, 12.333333333333334)
(4, 11.799999999999999)
1.0
0.0
0.0
0.0
(1, 1.833431952662722)
(4, 10.777727272727274)
0.0
(3, 0.41056603773584915)
0.0
(0, 0.3982142857142857)
(4, 11.73235294117647)
(3, 0.45099999999999996)
0.0
(0, 0.3075)
(0, 0.295)
0.0
1.0
0.0
(1, 1.1999999999999997)
(1, 0.975)
1.0
0.0
0.0
(4, 11.381818181818183

In [254]:
test= np.array([1,2])

In [255]:
np.where(test == max(test))[0][0]

1

In [256]:
target_names = np.array(target_names)


In [257]:
data = np.array((['sunny','sunny','sunny','sunny','sunny', 'rainy','rainy','rainy', 'cloudy','cloudy','cloudy'], [1,1,1,1,0,0,1,1,0,0,1]), dtype=str)

In [258]:
np.unique(data[0], return_counts=True)

(array(['cloudy', 'rainy', 'sunny'], dtype='<U6'),
 array([3, 3, 5], dtype=int64))

In [259]:
data
#data[data[0]!='sunny']

array([['sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'rainy', 'rainy',
        'rainy', 'cloudy', 'cloudy', 'cloudy'],
       ['1', '1', '1', '1', '0', '0', '1', '1', '0', '0', '1']],
      dtype='<U6')

In [260]:
df = pd.read_csv('cars.csv')
df

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year,brand
0,14.0,8,350,165,4209,12,1972,US.
1,31.9,4,89,71,1925,14,1980,Europe.
2,17.0,8,302,140,3449,11,1971,US.
3,15.0,8,400,150,3761,10,1971,US.
4,30.5,4,98,63,2051,17,1978,US.
...,...,...,...,...,...,...,...,...
256,17.0,8,305,130,3840,15,1980,US.
257,36.1,4,91,60,1800,16,1979,Japan.
258,22.0,6,232,112,2835,15,1983,US.
259,18.0,6,232,100,3288,16,1972,US.


In [261]:
data = df.to_numpy()
data

array([[14.0, 8, '350', ..., 12, 1972, ' US.'],
       [31.9, 4, '89', ..., 14, 1980, ' Europe.'],
       [17.0, 8, '302', ..., 11, 1971, ' US.'],
       ...,
       [22.0, 6, '232', ..., 15, 1983, ' US.'],
       [18.0, 6, '232', ..., 16, 1972, ' US.'],
       [22.0, 6, '250', ..., 15, 1977, ' US.']], dtype=object)

In [262]:
labels = data[:, -1]
data = data[:, :-1] 

In [271]:
data

array([[14.0, 8, 350, ..., 4209, 12, 1972],
       [31.9, 4, 89, ..., 1925, 14, 1980],
       [17.0, 8, 302, ..., 3449, 11, 1971],
       ...,
       [22.0, 6, 232, ..., 2835, 15, 1983],
       [18.0, 6, 232, ..., 3288, 16, 1972],
       [22.0, 6, 250, ..., 3353, 15, 1977]], dtype=object)

In [267]:
data[data[:, 2] == ' '] = 0 

In [268]:
for x in data[:,2]:
    x = int(x)

In [269]:
data[:, 2] = data[:, 2].astype(np.int)

C:\Users\Einar\AppData\Local\Temp\ipykernel_12376\1348596518.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data[:, 2] = data[:, 2].astype(np.int)


In [272]:
np.unique(labels, return_counts=True)

(array([' Europe.', ' Japan.', ' US.'], dtype=object),
 array([ 48,  51, 162], dtype=int64))

In [317]:
tree_3 = Tree()
tree_3 = learn(data, labels, impurity_measure='entropy', pruning='True')

In [318]:
impurity(labels[:70])

1.4013498193719667

In [319]:
accuracy(data, labels, tree_3)

0.9425287356321839

In [320]:
predict(data[0], tree_3)

' US.'

In [321]:
labels[0]

' US.'